In [ ]:
import re
import requests
import math
from lxml import html
from collections import Counter
from string import punctuation as enPunc
from zhon.hanzi import punctuation as zhPunc

import pandas as pd
import numpy as np
from elasticsearch import Elasticsearch
from bs4  import BeautifulSoup
from LAC import LAC
from opencc import OpenCC

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
"""
init: 使用者點選想看的網址，回傳該網址的rank
1. 從rank抓出網址
2. 將該文章進行命名實體標註、把不必要的詞刪除
3. 計算tfidf，萃取文章前20關鍵詞
"""

In [382]:
# 測試的網頁文章
url = ['https://www.bbc.com/zhongwen/trad/chinese-news-54160466']  # Search 1
# url = ['http://www.bast.net.cn/art/2019/12/25/art_23348_448376.html'] # Search 2

## 1. 資料處理與特徵萃取

### 資料處理：爬蟲、資料清洗、分詞、命名實體辨識

In [2]:
# data preprocessing
def multi_repl(input_str):
    trim = ['\n', '\r', '\t', '\xa0', ' '] + list(enPunc)+list(zhPunc) ## 網頁亂碼+英文標點+中文標點 (無數字)
    trim_dict = {pat:'' for pat in trim}
    for k in trim_dict:
        input_str = input_str.replace(k, trim_dict[k])  ## 把stop words改為空白
    return input_str

def getFinalText(google_search_rs):
    html_text = []
    getLink = []
    for idx, link in enumerate(google_search_rs):
        getLink.append((idx+1, link))
        print(idx +1, link)
        if 'pdf' in link or 'PDF' in link or 'download' in link or 'Download' in link:
            html_text.append('無結果')
            continue
        try:
            resp = requests.get(link, headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36'})
            soup = BeautifulSoup(resp.content, 'html.parser')
            a_tag = soup.find_all('a')
            for a in a_tag:
                a.decompose() 
            html_text.append(soup.get_text()) 
        except Exception as e:
            print(e)
            html_text.append('無結果')               
    
    # 繁轉簡
    cc = OpenCC('tw2sp')
    temp = [cc.convert(texts) for texts in html_text]

    # 只保留漢字、數字                 *[大寫英文字(\u0041-\u005a)]
    final_text = [re.sub(u"([^\u4e00-\u9fa5\u0030-\u0039])","", texts) for texts in temp]

    # 去除停用詞 (先去除還是等到分詞時再去除)
#     final_text = [multi_repl(texts) for texts in temp1]
    
    # 紀錄資訊量少的文章 (文字少於400者)
#     lessInfo = [i+1 for i in range(len(final_text)) if len(final_text[i]) < 400]
    
    return final_text, getLink

In [3]:
# 計算tfidf需要全部搜尋結果的詞頻，所以要跑一次全部的文章分詞
google_search_rs = pd.read_excel('data/search1.xlsx').url.to_list() 
all_docs, all_links = getFinalText(google_search_rs)
all_docs

1 https://zh.wikipedia.org/zh-tw/%E6%B7%B1%E5%9C%B3%E6%8C%AF%E5%8D%8E%E6%95%B0%E6%8D%AE%E4%BF%A1%E6%81%AF%E6%8A%80%E6%9C%AF
2 https://www.bbc.com/zhongwen/trad/chinese-news-54160466
3 https://www.bbc.com/zhongwen/simp/chinese-news-54160466
4 https://www.cna.com.tw/news/firstnews/202009220120.aspx
5 https://www.voachinese.com/a/zhenhua-data-leak-20200915/5584841.html
6 https://www.abc.net.au/chinese/2020-09-14/chinese-data-leak-linked-to-military-names-australians/12661676
7 https://technews.tw/2020/10/03/personal-details-of-millions-around-world-gathered-by-china-tech-company/
8 https://www.storm.mg/article/3037830?page=1
9 https://www.indiavsdisinformation.com/zh/20200930/zhenhua-data-leak-is-china-eyeing-australian-technology-
10 https://www.rfa.org/mandarin/yataibaodao/junshiwaijiao/jt-09142020110020.html
11 https://www.zhipin.com/gongsi/4ffe07f7ae8dff031Hd53Nu1Fg~~.html
12 https://global.udn.com/global_vision/story/8662/4858718
13 https://www.tianyancha.com/company/3179660979
14 ht

['深圳振华数据信息技术维基百科自由的百科全书深圳振华数据信息技术维基百科自由的百科全书深圳振华数据信息技术有限公司是一家成立于2017年位于的技术公司该公司除深圳外在北京设有办公室在全球200多个国家和地区设立数据处理中心其宣称的使用是集成全球开源数据助力中华民族伟大复兴首席行政官合作伙伴有致力于军民融合的公司及持有企业的控股子公司2018年起申请专利涉及新闻及信息抓取数据及视频处理及社交媒体操控2019年9月注册了仿真社交媒体交互的专利收集个人信息事件2020年9月13日曾在深圳工作的美国学者原经济学教授鲍尔丁揭露称该公司通过设立巨大境外个人数据库为中国军方收集了世界各地240万人的个人数据以此首次证实中国大量搜集境外个人数据以影响和控制外国事务的猜测该企业服务对象包括和该企业收集的信息涉及出生日期地址婚姻状况照片政治属性亲属及社交帐户如和也包括新闻及犯罪记录企业不端行为已收集信息包括35000名澳大利亚人虽然许多信息收集自公开来源但似乎也有信息来自私密的银行记录工作申请表和心理状况记录另有部分数据可能来源于振华公司在回应英国采访时称相关报导严重失实该公司为私营企业与中国政府和军方无关信息安全作家杰里米柯克认为振华收集公开数据的行为与一般的社交平台或广告公司无异世界范围内有许多其他公司提供类似的服务参考数据2020091420200914黎堡美国之音202009142020091420200914202009142020091420200914202009142020091420200916取自导览菜单个人工具没有登录命名空间台湾正体查看更多搜索导航说明工具打印导出其他语言本页面最后修订于2021年3月30日星期二0633本站的全部文本在之条款下提供附加条款亦可能应用请参阅和维基百科标志是的注册商标维基是维基媒体基金会的商标维基媒体基金会是按美国国内税收法5013登记的',
 '深圳振华数据如何看待中国国企监控数据库外泄的消息中文分类深圳振华数据如何看待中国国企监控数据库外泄的消息李翰文中文2020年9月16日图像来源图像加注文本振华数据的数据库据报覆盖约240万人包括多国军方政界商界等人的数据多家英国印度澳大利亚等地的传媒早前报道一家中国国有公司利用自动化技术在网络上搜集全球各国政界商界等知名人士的数据范围更覆盖他们的亲友和关系密切人士引起外界对中国情报搜集的

In [384]:
# 欲取得關鍵字的文章，需要取得詞頻
current_doc, currentLink = getFinalText(url)
current_doc

1 https://www.bbc.com/zhongwen/trad/chinese-news-54160466


['深圳振华数据如何看待中国国企监控数据库外泄的消息中文分类深圳振华数据如何看待中国国企监控数据库外泄的消息李翰文中文2020年9月16日图像来源图像加注文本振华数据的数据库据报覆盖约240万人包括多国军方政界商界等人的数据多家英国印度澳大利亚等地的传媒早前报道一家中国国有公司利用自动化技术在网络上搜集全球各国政界商界等知名人士的数据范围更覆盖他们的亲友和关系密切人士引起外界对中国情报搜集的关注澳洲广播公司英国每日电讯报印度快报等媒体周一9月14日发稿引述一个从在越南富布赖特大学任教的美国学者包定又译克里斯托弗巴尔德或巴尔丁那里取得的数据库指这个由一家位于中国深圳的振华数据公司创建的数据库收录了各地的政客法官军官商人甚至影视明星的数据报道引起各界关注一些评论认为这个数据库显示中国官方搜集和整理情报的能力但同时有专家指出数据库内收录的绝大多数都是公开数据而编写搜集网络上公开数据的电脑程序也不是什么难事质疑事件是否如外界所言这样敏感图像来源图像加注文本数据库收藏英国首相约翰逊等多国元首的数据但大多为公开数据振华数据是什么样的公司这个数据库称为海外核心信息数字库由一家称为振华数据的公司开发振华数据的网站目前已被关闭但中文透过网站存盘服务找到公司的网站网站的介绍指这个数据库有四部份包括人物库机构库信息库和关系库网站显示数据库存有约240万个人物的数据其中网站介绍人物库包含全球军政商科技传媒民间组织等领域的领袖及核心人物并包括他们在推特脸书领英和博客平台的数据拼合成个人文件机构库就包含军政商科技传媒民间组织等领域的核心机构信息库提供它们相关的新闻信息这个数据库还会纪录各个人物和机构之间的关系图像来源图像加注文本振华的网站显示数据库收藏约240万个人物的数据网站目前已经关停编写这个数据库的振华数据母公司是振华电子集团总部设于贵州省贵州省国资委和同为国有企业的中国电子是最大股东根据香港媒体报道公司行政总裁是王雪峰他曾经在美国国际商业机器公司工作也曾经在微信上表示支持利用数据发动信息战中文透过振华数据的网站找到一位姓孙的联系人对方形容有传媒报道无中生有但记者追问详情后对方就匆忙挂线中国官方至今仍未就振华数据相关的报道作出表态印度快报引述中国驻新德里一名外交消息人士指中国未曾也不会要求公司或个人收集或提供数据但没有说明中国官方与振华数据的关系这个数据库是什么向传媒披露这个数据库的美国

In [4]:
def stopwords():
    # Load stop words
    f = open('/Users/csti-user/Downloads/NER/stopwords-master/baidu_stopwords.txt','r')
    k = f.readlines()
    f.close()
    stopwords = [lines.strip('\n') for lines in k]

    # 建構stopwords
    trim = ['\n', '\r', '\t', '\xa0'] + list(enPunc)+list(zhPunc) ## 網頁亂碼+英文標點+中文標點
    trim = trim + stopwords  # list of all stopwords
    return trim

# 命名實體辨識、計算詞頻 
def word_segments(final_text):
    lac = LAC(mode='lac')  
    lac.load_customization('custom_dict.txt')  # 自行增加字典
    trim = stopwords()
    df_all = pd.DataFrame(columns=['words', 'ner', 'doc'])
    cnt = []
    for i in range(len(final_text)):
        ner_model = lac.run(final_text[i])
        for k in range(len(ner_model[0])):
            if ner_model[0][k] not in trim and len(ner_model[0][k])>1:
                if ner_model[1][k] == 'PER' or ner_model[1][k] == 'ORG' or ner_model[1][k] == 'nz' or ner_model[1][k] == 'an' or ner_model[1][k] == 'vn':
                    if all(str.isdigit() for str in ner_model[0][k]) == False:
                        if any(str.isdigit() for str in ner_model[0][k]) == False:
                            df_all = df_all.append({'words': ner_model[0][k], 'ner': ner_model[1][k], 'doc': i+1}, ignore_index=True)
        cnt.append(Counter(df_all[df_all.doc == i+1]['words']))
    return cnt

In [5]:
# 所有文本的詞頻
all_docs_cnt = word_segments(all_docs)
all_docs_cnt

[Counter({'振华数据': 3,
          '信息技术有限公司': 1,
          '中华民族': 1,
          '复兴': 1,
          '合作': 1,
          '控股': 1,
          '申请': 1,
          '仿真': 1,
          '收集': 1,
          '学者': 1,
          '鲍尔丁': 1,
          '数据库': 1,
          '影响': 1,
          '猜测': 1,
          '服务': 2,
          '出生': 1,
          '犯罪': 1,
          '公开': 1,
          '振华': 2,
          '公司': 1,
          '相关': 1,
          '报导': 1,
          '中国政府': 1,
          '杰里米柯克': 1,
          '参考': 1,
          '命名': 1,
          '搜索': 1,
          '导航': 1,
          '附加': 1,
          '参阅': 1}),
 Counter({'振华数据': 24,
          '数据库': 36,
          '李翰文': 1,
          '中国国有公司': 1,
          '自动化': 1,
          '澳洲广播公司': 4,
          '电讯报印度快报': 1,
          '越南富布赖特大学': 2,
          '学者': 4,
          '包定': 3,
          '克里斯托弗巴尔德': 3,
          '巴尔丁': 1,
          '公司': 1,
          '评论': 2,
          '专家': 2,
          '质疑': 1,
          '约翰逊': 3,
          '数字库': 2,
          '存盘': 1,
          '服务':

In [386]:
# 欲查看關鍵字的文本的詞頻
current_doc_cnt = word_segments(current_doc)
current_doc_cnt

[Counter({'振华数据': 24,
          '数据库': 36,
          '李翰文': 1,
          '中国国有公司': 1,
          '自动化': 1,
          '澳洲广播公司': 4,
          '电讯报印度快报': 1,
          '越南富布赖特大学': 2,
          '学者': 4,
          '包定': 3,
          '克里斯托弗巴尔德': 3,
          '巴尔丁': 1,
          '公司': 1,
          '评论': 2,
          '专家': 2,
          '质疑': 1,
          '约翰逊': 3,
          '数字库': 2,
          '存盘': 1,
          '服务': 3,
          '介绍': 2,
          '人物库': 2,
          '机构库': 2,
          '信息库': 2,
          '关系库': 1,
          '网站': 1,
          '推特': 3,
          '脸书': 2,
          '领英': 1,
          '拼合成': 1,
          '振华': 2,
          '母公司': 1,
          '电子集团': 1,
          '贵州省贵州省国资委': 1,
          '王雪峰': 2,
          '美国国际商业机器公司': 1,
          '微信': 1,
          '信息战': 2,
          '印度快报': 1,
          '澳大利亚网络安全公司': 1,
          '运行': 2,
          '罗伯特波特': 1,
          '协助': 1,
          '相关': 2,
          '莫里森': 1,
          '莫迪': 1,
          '升迁': 1,
          '中国人民解放军': 1,
         

## 2. 關鍵資訊擷取

### 使用TF-IDF進行關鍵字擷取

In [387]:
#计算count_list有多少个文件包含word
def n_containing(word, count_list):
    return sum(1 for count in count_list if word in count)

# 计算tf
def tf(word, count):
    return count[word] / sum(count.values())

# 计算idf
def idf(word, count_list):
    return math.log(len(count_list) / (n_containing(word, count_list)))  

In [177]:
# 取得欲查詢關鍵字的文本index
def getIdx(url, getLink):
    df = pd.DataFrame(getLink, columns=['idx', 'link'])
    for i in range(len(df)):
        if df.iloc[i,1] == url[0]:
            return i

In [388]:
# 取得該文本n(20)個關鍵字
wd = list(current_doc_cnt[0].keys())
df_tfidf = pd.DataFrame(columns=['word','tfidf'])
for i, count in enumerate(all_docs_cnt):
    if i == getIdx(url, all_links):
        for word in count:
#             if word in wd and len(word)>1:
            df_tfidf = df_tfidf.append({'word':word,'tfidf': tf(word,count)*idf(word,all_docs_cnt)},
                                            ignore_index=True)
df_tfidf = df_tfidf.sort_values(by=['tfidf'], ascending=False).reset_index()
df_tfidf = df_tfidf.drop(columns=['index'])  ## 程式產出結果

In [389]:
# Topic 1, Search 1: BBC文章 
df_tfidf[:20]

,word,tfidf
0,数据库,0.069784
1,澳洲广播公司,0.054907
2,克里斯托弗巴尔德,0.052482
3,柯克,0.052482
4,运行,0.042522
5,信息库,0.042522
6,访问,0.041180
7,包定,0.037542
8,罗杰卡尔,0.034988
9,约翰逊,0.034570


In [381]:
# Topic 1, Search 2: 北京情報學會 
df_tfidf[:20]

,word,tfidf
0,北京科学技术情报学会,0.215341
1,创新,0.215341
2,与会,0.143560
3,交流,0.118124
4,北京市科学技术协会,0.107670
5,中国科协,0.071780
6,北京市科协,0.071780
7,吴晨生,0.071780
8,戴国强,0.071780
9,对话,0.071780
